<a href="https://colab.research.google.com/github/YassGan/RTG-SLAM/blob/main/SPLATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making sure that we are using a GPU

In [1]:
!nvidia-smi

Mon Mar 24 00:27:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:

!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit --no-drm --no-man-page
import os
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:/usr/lib64-nvidia'
!nvcc --version  # Should show CUDA 11.8

--2025-03-24 00:47:01--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.43.51.10, 23.43.51.15
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.43.51.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   184MB/s    in 40s     

2025-03-24 00:47:41 (104 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [11]:
# Uninstall existing torch versions
!pip uninstall torch torchvision torchaudio -y

# Install torch 2.0.1 with CUDA 11.8 support
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 522.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0


# Python downgrading


In [12]:

!apt update
!apt install python3.10 python3.10-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
!python3.10 --version  # Should show 3.10.x

!sudo update-alternatives --remove-all python3  # Clear existing alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3  # If prompted, select 3.10
!python3 --version



Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Pac

#Pytorch with cuda

# Verification of the versions

In [13]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.version.cuda)        # Should be 11.3 (from PyTorch)
print(torch.cuda.get_device_name(0))  # Should show GPU

True
11.8
Tesla T4


In [14]:
##making sure that we are always using the GPU and not a CPU
!nvidia-smi

Mon Mar 24 00:57:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
# Verify installation
import torch
print(torch.__version__)  # Should show 2.0.1+cu118
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should show 11.8

2.0.1+cu118
True
11.8


In [16]:
# Clone SplaTAM
!git clone https://github.com/spla-tam/SplaTAM.git
%cd SplaTAM

# Install requirements with python3.10
!python3.10 -m pip install -r requirements.txt
!python3.10 -m pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

Cloning into 'SplaTAM'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 210 (delta 93), reused 43 (delta 43), pack-reused 79 (from 2)
Receiving objects: 100% (210/210), 14.35 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/SplaTAM
  Cloning https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git (to revision cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110) to /tmp/pip-req-build-upbgp4oe
  Running command git clone --filter=blob:none --quiet https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git /tmp/pip-req-build-upbgp4oe
  Running command git rev-parse -q --verify 'sha^cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110'
  Running command git fetch -q https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110
  Resolved https://github.com/JonathonLuiten/diff-gaussian-rasterization-w

In [17]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [18]:
# Install build tools
!apt-get update
!apt-get install -y build-essential cmake ninja-build

# Verify build tools
!g++ --version
!cmake --version
!ninja --version

# Set CUDA environment (already installed, just ensuring PATH)
import os
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:/usr/lib64-nvidia'
!nvcc --version  # Should show CUDA 11.8

# Uninstall any existing torch versions
!pip uninstall torch torchvision torchaudio -y

# Install PyTorch with CUDA 11.8
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Clone SplaTAM
!git clone https://github.com/spla-tam/SplaTAM.git
%cd SplaTAM

# Install diff-gaussian-rasterization-w-depth
!pip install git+https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git@cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110

# Install tiny-cuda-nn
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

# Install remaining requirements
!pip install -r requirements.txt

# Verify PyTorch installation
import torch
print(torch.__version__)  # Should show 2.0.1+cu118
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should show 11.8

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

Cloning into 'SplaTAM'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 210 (delta 93), reused 41 (delta 41), pack-reused 79 (from 2)
Receiving objects: 100% (210/210), 14.35 MiB | 17.49 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/SplaTAM/SplaTAM
  Cloning https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git (to revision cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110) to /tmp/pip-req-build-2wbq13pj
  Running command git clone --filter=blob:none --quiet https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git /tmp/pip-req-build-2wbq13pj
  Running command git rev-parse -q --verify 'sha^cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110'
  Running command git fetch -q https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110
  Resolved https://github.com/JonathonLuiten/diff-gaussian-rasteri